In [ ]:
import glob
import nbformat
from nbformat.v4 import new_code_cell
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import re
import textwrap
import shutil
import json
import pandas as pd
from pathlib import Path
from datetime import datetime


In [ ]:
fpaths = glob.glob('../submissions/case-study-03*/*.ipynb')
fpaths[:5]

In [ ]:
df = pd.DataFrame({
    'filename': pd.Series(dtype='str'),
    'grading_finished_at': pd.Series(dtype='str'),
    'grading_duration_in_seconds': pd.Series(dtype='int'),
    'learner_score': pd.Series(dtype='float'),
    'total_available': pd.Series(dtype='float'),
    'num_test_cases': pd.Series(dtype='int'),
    'num_passed_cases': pd.Series(dtype='int'),
    'num_failed_cases': pd.Series(dtype='int'),
    'summary': ''
})
df

In [ ]:
from lambdagrader import *

graded_results = []

for notebook_path in fpaths[:3]:
    try:
        print('=============================')
        nb = nbformat.read(notebook_path, as_version=4)
        
        for cell in nb.cells:
            test_case_metadata = extract_test_case_metadata(cell.source)

            if test_case_metadata:
                cell.source = convert_to_grader_code(cell.source)

            converted_notebook_path = notebook_path.replace('.ipynb', '-graded.ipynb')
            graded_notebook_filename = Path(converted_notebook_path).name

        add_scripts_to_notebook(nb)

        with open(converted_notebook_path, mode='w', encoding='utf-8') as f:
            nbformat.write(nb, f)

        nb = nbformat.read(converted_notebook_path, as_version=4)
        
        ep = ExecutePreprocessor(
            timeout=600,
            kernel_name='python3',
            allow_errors=True
        )
        ep.preprocess(nb)
        print(f'Running {converted_notebook_path} successful')
        
        graded_result_json_path = notebook_path.replace('.ipynb', '-result.json')
        shutil.move('lambdagrader-result.json', graded_result_json_path)
        print(f'Stored graded result as JSON to {graded_result_json_path}')
        
        with open(graded_result_json_path, mode='r') as f:
            graded_result = json.load(f)
            
        summary = ''
        summary += f"File: {graded_notebook_filename}\n"
        summary += f"Score: {graded_result['learner_score']} out of {graded_result['total_available']}\n"
        summary += f"Passed {graded_result['num_passed_cases']} out of {graded_result['num_test_cases']} test cases\n"
        summary += f"Grading took {graded_result['grading_duration_in_seconds']} seconds\n\n"
        summary += 'Test Case Summary\n'
        
        for o in graded_result['results']:
            summary += f"{o['test_case_name']} {'passed' if o['pass'] else 'failed'}: {o['points']} out of {o['available_points']} points\n"
            
            if not o['pass']:
                summary += f"[Autograder Output]\n{o['message']}\n\n"
                
        result_summary = {
            'filename': graded_notebook_filename,
            'grading_finished_at': graded_result['grading_finished_at'],
            'grading_duration_in_seconds': graded_result['grading_duration_in_seconds'],
            'learner_score': graded_result['learner_score'],
            'total_available': graded_result['total_available'],
            'num_test_cases': graded_result['num_test_cases'],
            'num_passed_cases': graded_result['num_passed_cases'],
            'num_failed_cases': graded_result['num_failed_cases'],
            'summary': summary
        }
        
        print(result_summary)
        graded_results.append(result_summary)
    except CellExecutionError as e:
        print(f'CellExecutionError on {notebook_path}')
        print('-----------------------------')
        print(e)
        
df_summary = pd.DataFrame(graded_results)

df_summary.to_csv(f"graded_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")